In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains #Mouse cursor
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

C:\Users\SOAdmin\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
firefox_driver_path = 'geckodriver.exe'
driver = webdriver.Firefox()
url = 'https://splunk.hh.polyu.hk:8000/en-US/app/so_iot/so_distance_light_timer_data_in_a_row?form.token_device=200&form.token_time.earliest=1637683200&form.token_time.latest=1645113600&form.minimum_distance=*&form.maximum_distance=*&form.minimum_illumination=*&form.maximum_illumination=*'
result = driver.get(url)

In [3]:
#login
username = driver.find_element_by_name('username')
username.send_keys("")
password = driver.find_element_by_name('password')
password.send_keys("")
button = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/form/fieldset/input[1]')
button.click()


<ipython-input-3-9699a397bc19>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username = driver.find_element_by_name('username')
<ipython-input-3-9699a397bc19>:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_name('password')
<ipython-input-3-9699a397bc19>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  button = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/form/fieldset/input[1]')


In [ ]:
html = driver.find_element_by_tag_name('html')#Scroll to the end of html
html.send_keys(Keys.PAGE_DOWN)
html.send_keys(Keys.PAGE_DOWN)
txpanel= driver.find_element_by_link_text('Event_Time') # Tx data panel Before downloading, move cursor to pop up menu to triger expect condition ->
action = ActionChains(driver) 
action.move_to_element(txpanel).perform() #Move cursor
WebDriverWait(driver, 1000000).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[3]/div[6]/div/div/div[2]/div/div[3]/div/a[2]'))).click()
filename = driver.find_element_by_name("fileName")
filename.send_keys("TxTest")
savebutton = driver.find_element_by_xpath('/html/body/div[5]/div/div[3]/a[2]')
savebutton.click()

In [ ]:
rwpanel = driver.find_element_by_link_text('Device_Name') # Raw data panel
html.send_keys(Keys.PAGE_DOWN)
action.move_to_element(rwpanel).perform()
WebDriverWait(driver, 1000000).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[3]/div[7]/div/div/div[2]/div/div[3]/div/a[2]'))).click()
filename = driver.find_element_by_name("fileName")#!! problem here, id diff every time
filename.send_keys("RwTest")
savebutton = driver.find_element_by_xpath('/html/body/div[5]/div/div[3]/a[2]')
savebutton.click()

In [4]:
def txdata (index, devicename):
    html = driver.find_element_by_tag_name('html')#Scroll to the end of html
    html.send_keys(Keys.PAGE_DOWN)
    html.send_keys(Keys.PAGE_DOWN)
    txpanel= driver.find_element_by_link_text('Event_Time') # Tx data panel Before downloading, move cursor to pop up menu to triger expect condition ->
    action = ActionChains(driver) 
    action.move_to_element(txpanel).perform() #Move cursor
    WebDriverWait(driver, 1000000).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[3]/div[6]/div/div/div[2]/div/div[3]/div/a[2]'))).click()
    filename = driver.find_element_by_name("fileName")
    filename.send_keys('BS'+str(index)+'_'+str(devicename)+'_t_'+'02052023'+'.csv')
    savebutton = driver.find_element_by_xpath('/html/body/div[5]/div/div[3]/a[2]')
    savebutton.click()
    
def rwdata (index, devicename):
    html = driver.find_element_by_tag_name('html')
    action = ActionChains(driver)
    html.send_keys(Keys.PAGE_DOWN)
    rwpanel = driver.find_element_by_link_text('Device_Name') # Raw data panel
    action.move_to_element(rwpanel).perform()
    WebDriverWait(driver, 1000000).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[3]/div[7]/div/div/div[2]/div/div[3]/div/a[2]'))).click()
    filename2 = driver.find_element_by_name("fileName")#!! problem here, id diff every time
    filename2.click()
    filename2.send_keys('BS'+str(index)+'_'+str(devicename)+'_r_'+'02052023'+'.csv')
    time.sleep(2)
    filename2.send_keys(Keys.ENTER)
    savebutton = driver.find_element_by_xpath('/html/body/div[5]/div/div[3]/a[2]')
    savebutton.click()

In [5]:
sidf=pd.read_csv('IoTListDispensed_2023-05-02.csv')
#sidf.set_index('record_id',inplace=True)
for index,row in sidf.iterrows():
    redirect='https://splunk.hh.polyu.hk:8000/en-US/app/so_iot/so_distance_light_timer_data_in_a_row?form.token_device='+str(row['devicename'])+'&form.token_time.earliest=1679155200&form.token_time.latest=1683043200&form.minimum_distance=*&form.maximum_distance=*&form.minimum_illumination=*&form.maximum_illumination=*'
    driver.get(redirect)
    time.sleep(180)
    #print('loading'+str(row['devicename']))
    try:
        txdata(row['record_id'],row['devicename'])
        rwdata(row['record_id'],row['devicename'])
    except:
        print(row,'error')
        continue

<ipython-input-4-d75610436495>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  html = driver.find_element_by_tag_name('html')#Scroll to the end of html
<ipython-input-4-d75610436495>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  txpanel= driver.find_element_by_link_text('Event_Time') # Tx data panel Before downloading, move cursor to pop up menu to triger expect condition ->
<ipython-input-4-d75610436495>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  filename = driver.find_element_by_name("fileName")
<ipython-input-4-d75610436495>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  savebutton = driver.find_element_by_xpath('/html/body/div[5]/div/div[3]/a[2]')
<ipython-input-4-d75610436495>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_e

record_id                              8
redcap_event_name    spectacle_blt_arm_3
bltsn                               A008
dispenseddate             23-10-21 13:30
devicename                           165
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3                     NaN
replacedate3                         NaN
replacedeviceid4                     NaN
replacedate4                         NaN
Unnamed: 13                          NaN
Name: 5, dtype: object error
record_id                             14
redcap_event_name    spectacle_blt_arm_3
bltsn                               A015
dispenseddate             18-10-21 14:30
devicename                           653
replacedeviceid1                     213
replacedate1              17-09-22 15:57
replacedeviceid2                     653
replacedate2              12-11-22 14:42
replacedeviceid3            

record_id                             60
redcap_event_name    spectacle_blt_arm_3
bltsn                               A060
dispenseddate             17-12-21 17:00
devicename                           177
replacedeviceid1                     261
replacedate1              09-02-22 16:00
replacedeviceid2                     214
replacedate2              29-05-22 10:33
replacedeviceid3                     177
replacedate3              02-12-22 14:21
replacedeviceid4                     NaN
replacedate4                         NaN
Unnamed: 13                          NaN
Name: 44, dtype: object error
record_id                             68
redcap_event_name    spectacle_blt_arm_3
bltsn                               A068
dispenseddate             03-12-21 16:00
devicename                           248
replacedeviceid1                     248
replacedate1              29-04-22 15:33
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3           

record_id                            114
redcap_event_name    spectacle_blt_arm_3
bltsn                               A114
dispenseddate             20-01-22 15:30
devicename                           268
replacedeviceid1       photometer broken
replacedate1              08-09-22 14:42
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3                     NaN
replacedate3                         NaN
replacedeviceid4                     NaN
replacedate4                         NaN
Unnamed: 13                        Blank
Name: 83, dtype: object error
record_id                            115
redcap_event_name    spectacle_blt_arm_3
bltsn                               B115
dispenseddate             27-01-22 16:16
devicename                           109
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3           

record_id                            136
redcap_event_name    spectacle_blt_arm_3
bltsn                               A136
dispenseddate             11-02-22 16:37
devicename                           116
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3                     NaN
replacedate3                         NaN
replacedeviceid4                     NaN
replacedate4                         NaN
Unnamed: 13                          NaN
Name: 103, dtype: object error
record_id                            137
redcap_event_name    spectacle_blt_arm_3
bltsn                               A137
dispenseddate             12-02-22 10:45
devicename                            82
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3          

...

record_id                            260
redcap_event_name    spectacle_blt_arm_3
bltsn                               A260
dispenseddate             19-11-22 12:00
devicename                           128
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3                     NaN
replacedate3                         NaN
replacedeviceid4                     NaN
replacedate4                         NaN
Unnamed: 13                          NaN
Name: 204, dtype: object error
record_id                            261
redcap_event_name    spectacle_blt_arm_3
bltsn                               B261
dispenseddate             25-11-22 16:07
devicename                           670
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3          

record_id                            277
redcap_event_name    spectacle_blt_arm_3
bltsn                               A277
dispenseddate             31-12-22 14:47
devicename                           609
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3                     NaN
replacedate3                         NaN
replacedeviceid4                     NaN
replacedate4                         NaN
Unnamed: 13                          NaN
Name: 221, dtype: object error
record_id                            281
redcap_event_name    spectacle_blt_arm_3
bltsn                                NaN
dispenseddate             14-01-23 14:00
devicename                           640
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3          

record_id                            306
redcap_event_name    spectacle_blt_arm_3
bltsn                               A306
dispenseddate             01-02-23 11:30
devicename                           213
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3                     NaN
replacedate3                         NaN
replacedeviceid4                     NaN
replacedate4                         NaN
Unnamed: 13                          NaN
Name: 245, dtype: object error
record_id                            308
redcap_event_name    spectacle_blt_arm_3
bltsn                               A308
dispenseddate             10-02-23 17:37
devicename                           179
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3          

record_id                            324
redcap_event_name    spectacle_blt_arm_3
bltsn                               B324
dispenseddate             05-03-23 14:47
devicename                           668
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3                     NaN
replacedate3                         NaN
replacedeviceid4                     NaN
replacedate4                         NaN
Unnamed: 13                          NaN
Name: 263, dtype: object error
record_id                            325
redcap_event_name    spectacle_blt_arm_3
bltsn                               A325
dispenseddate             09-03-23 17:04
devicename                            57
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3          

record_id                            349
redcap_event_name    spectacle_blt_arm_3
bltsn                               A349
dispenseddate             21-03-23 13:41
devicename                           480
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3                     NaN
replacedate3                         NaN
replacedeviceid4                     NaN
replacedate4                         NaN
Unnamed: 13                          NaN
Name: 287, dtype: object error
record_id                            350
redcap_event_name    spectacle_blt_arm_3
bltsn                               B350
dispenseddate             24-03-23 13:42
devicename                           189
replacedeviceid1                     NaN
replacedate1                         NaN
replacedeviceid2                     NaN
replacedate2                         NaN
replacedeviceid3          

In [ ]:
sidf=pd.read_csv('RIF_IoTListDispensed_20220218.csv')
#sidf.set_index('record_id',inplace=True)
for index,row in sidf.iterrows():
    print (row['devicename'],row['record_id'])
'''for iot in sidf['devicename']:
    subjectid='BS'+ sidf['record_id']
    print(subjectid)'''

In [ ]:
filename2 = driver.find_element_by_name("fileName")
filename2.send_keys('BS_r_18032022')